# Modello di deep learning per predizione dropout scolastico su dati INVALSI
Progetto del corso di **Intelligenza Artificiale**, A.A. 2020/2021

**LM Informatica**, **Alma Mater Studiorum - Università di Bologna**

Realizzato da:
- Marco Ferrati, matr. 983546
- Michele Perlino, matr. 983733
- Tommaso Azzalin, matr. 985911

# 1 - Setup dell'ambiente

## 1.1 - Installazione delle librerie necessarie

In [ ]:
!pip3 install --no-cache-dir -r requirements.txt

## 1.2 - Import delle librerie fondamentali per l'analisi dei dati

In [ ]:
import re
from beautifultable import BeautifulTable

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import IntegerLookup
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTENC
from livelossplot import PlotLossesKeras

Com'è ben noto, il machine learning è un ambito in cui la sperimentazione occupa un ruolo molto importante: una volta consolidati i fondamenti teorici, abbiamo dovuto sperimentare innumerevoli architetture neurali differenti, tracciando di volta in volta le performance raggiunte.\
Al fine di rendere il codice del notebook quanto più flessibile, abbiamo creato il file Python `config.py` contenente la definizione degli iperparametri della rete; in dettaglio, in questo script, per ogni iperparametro, si verifica l'esistenza di una variabile d'ambiente col medesimo nome: in caso positivo, si assegna il valore di quest'ultima, mentre, in caso negativo, si assegna un valore di default.

Ad esempio:

`LEARNING_RATE = float(getenv(key="LEARNING_RATE", default="0.001"))`

Per impostare variabili d'ambiente in maniera batch, abbiamo creato dei file con estensione `.env`:
- se si opera da PowerShell, è sufficiente dare il comando `Set-PsEnv` (installabile mediante comando `Install-Module -Name Set-PsEnv`) per definire tutte le variabili d'ambiente presenti nel file `.env` che si trova nella directory corrente;
- se si opera da una shell Unix, si può impostare una variabile d'ambiente alla volta con `export nome_variabile=valore`: il contenuto di una variabile può essere controllato con `echo $nome_variabile`.

In [ ]:
import src.config as cfg

In [ ]:
print("Configuration")
if cfg.check_config() > 0:
    print("The configuration is incorrect. Please fix it before continuing otherwise you could encounter issues while working with this notebook.")

cfg.print_config()

All'interno della directory `src`, abbiamo creato altri due file Python (`mapping_domande_ambiti_processi.py` e `column_converters.py`) contenente del codice che si è voluto separare da quello del notebook, per questioni di ordine e leggibilità. 

In [ ]:
from src.mapping_domande_ambiti_processi import MAPPING_DOMANDE_AMBITI_PROCESSI
from src.column_converters import COLUMN_CONVERTERS

Nella cartella `src`, di cui sopra, sono presenti altri due file Python `invalsi.py` e `save_plots.py`. Il primo contiene del codice equivalente a quello presente in questo notebook, ma che può essere eseguito da linea di comando come un qualsiasi script Python; la sua creazione si è resa necessaria per l'esecuzione degli script attraverso i job avviati sul Cluster HPC del DISI, descritto nella sezione [Esecuzione su Cluster HPC](#cluster). Il secondo contiene del codice per memorizzare dei grafici sotto forma di file immagine contenenti gli andamenti delle metriche che permettono di valutare il funzionamento di un'architettura neurale.

## 1.3 - Import del dataset originale

In [ ]:
"""
Eseguire per: analisi dataset
"""
original_dataset = pd.read_csv(cfg.ORIGINAL_DATASET, sep=';', converters=COLUMN_CONVERTERS)

# 2 - Exploratory Data Analysis del dataset originale
Il codice che segue realizza il processo EDA, acronimo che sta per *Exploratory Data Analysis*, al fine di scandagliare i punti di attenzione del dataset a disposizione.

In [ ]:
original_dataset.info()

Come restituito dalla funzione `info()`, il dataset presenta 342226 righe e 104 colonne.

In [ ]:
original_dataset.columns

Il dataset contiene svariate informazioni relative a studenti che hanno ultimato il ciclo di studi delle superiori: l'obiettivo del progetto è la progettazione e implementazione di un classificatore capace di predire, sulla base dei risultati conseguiti al test INVALSI di terza media, quali studenti registreranno un *dropout*.\
Il concetto di *dropout* può essere declinato su due livelli:
- **implicito**: si dice che lo studente ha registrato un dropout implicito nel caso in cui non abbia acquisito le minime conoscenze e competenze, per cui presenta lacune formative;
- **esplicito**: si dice che lo studente ha registrato un dropout esplicito nel caso in cui non abbia conseguito il diploma.

## 2.1 - Ricerca di colonne con alte percentuali di valori nulli

In [ ]:
print("Columns with high null values percentages:")

table = BeautifulTable()
table.columns.header = ["", "Type","Ratio null values"]

for col in original_dataset.columns :
    ratio_null_values = original_dataset[col].isnull().mean().round(3)
    if ratio_null_values > 0:
        table.rows.append([col, str(original_dataset[col].dtypes), ratio_null_values])
table.rows.append(['LIVELLI', str(original_dataset['LIVELLI'].dtypes), original_dataset['LIVELLI'].isnull().mean().round(3)])
table.rows.append(['DROPOUT', str(original_dataset['DROPOUT'].dtypes), original_dataset['DROPOUT'].isnull().mean().round(3)])
        
table.columns.alignment = BeautifulTable.ALIGN_LEFT
table.set_style(BeautifulTable.STYLE_SEPARATED)
table.rows.sort('Ratio null values')
print(table)

In [ ]:
columns_high_ratio_null_values = ["codice_orario", "PesoClasse", "PesoScuola", "PesoTotale_Matematica"]
columns_low_ratio_null_values = [
    "voto_scritto_ita",  # 0.683
    "voto_scritto_mat",  # 0.113
    "voto_orale_ita",  # 0.683
    "voto_orale_mat"  # 0.114
]

## 2.2 - Ricerca colonne con un numero basso di valori distinti 

In [ ]:
print("Columns with unique values:")

table = BeautifulTable()
table.columns.header = ["", "Ratio distinct values"]

for col in original_dataset.columns:
    ratio_unique_vals = round(original_dataset[col].nunique() / len(original_dataset), 3)
    if ratio_unique_vals  > 0.1 :
        table.rows.append([col, ratio_unique_vals]) 

table.columns.alignment = BeautifulTable.ALIGN_LEFT
table.set_style(BeautifulTable.STYLE_SEPARATED)
table.rows.sort('Ratio distinct values')
print(table)

Come era prevedibile, `Unnamed: 0` (corrispondente alla colonna con l'indice della riga) e `CODICE_STUDENTE` presentano una proporzione di valori distinti sul totale delle righe pari a 1, in quanto trattasi di indici/codici che identificano univocamente gli studenti. Queste due colonne possono essere eliminate, dato che non portano alcuna informazione che possa aiutare a ravvisare relazioni tra gli studenti.

In [ ]:
columns_with_unique_values = ["Unnamed: 0", "CODICE_STUDENTE"]

## 2.3 - Ricerca colonne con un singolo valore

In [ ]:
print("Columns with just one value:")
for col in original_dataset.columns:
    unique_vals = original_dataset[col].nunique()
    if unique_vals == 1:
        print(col)

Similmente, queste colonne possono essere eliminate in quanto non distinguono in alcuna maniera gli studenti.

In [ ]:
columns_with_just_one_value = ["macrotipologia", "livello"]

## 2.4 - Rimozione delle colonne superflue

In [ ]:
"""
Eseguire per: analisi dataset
"""
cleaned_original_dataset: pd.DataFrame = original_dataset.drop(
    columns_high_ratio_null_values + 
    columns_with_unique_values + 
    columns_with_just_one_value, 
    axis=1
)
cleaned_original_dataset.to_csv(cfg.CLEANED_DATASET)

In [ ]:
"""
Eseguire per: analisi dataset
Attenzione: inutile eseguire se si è eseguita la precedente cella.
"""
cleaned_original_dataset = pd.read_csv(cfg.CLEANED_DATASET)

In [ ]:
if "Unnamed: 0" in cleaned_original_dataset.columns:
    cleaned_original_dataset.drop("Unnamed: 0", axis=1, inplace=True)

## 2.5 - Generalizzazione dataset per supportare altre coorti di studenti
Addestrando la rete con le domande specifiche di un test INVALSI di uno specifico anno non si ottiene un classificatore riutilizzabile per coorti successive (le domande cambiano ogni anno). Pertanto, abbiamo pensato di mappare le feature inerenti alle domande in uno spazio più generico che permetta di cogliere la loro semantica piuttosto che la loro rappresentazione letterale; mediante le griglie di correzione fornite ai docenti, abbiamo notato che ogni domanda è caratterizzata da uno o più ambiti e processi: questa corrispondenza ha ispirato la trasformazione dello spazio di rappresentazione delle domande che riportiamo di seguito. 

Nel file `mapping_domande_ambiti_processi.py` posizionato nella cartella `src` abbiamo creato una mappa (`dict` in Python) le cui chiavi sono le domande e i valori gli ambiti e i processi corrispondenti. Dopo averlo importato nel notebook corrente, abbiamo estratto i distinti ambiti e processi per poi calcolarne il numero di domande che caratterizzano.

In [ ]:
list_ambiti_processi = [AP for val in MAPPING_DOMANDE_AMBITI_PROCESSI.values() for AP in val]
ambiti_processi = set(list_ambiti_processi)
conteggio_ambiti_processi = {AP: list_ambiti_processi.count(AP) for AP in ambiti_processi}   

Abbiamo proceduto con l'aggiungere al dataset originale colonne recanti il nome degli ambiti e processi, inizializzate a 0 per tutte le righe.

In [ ]:
"""
Eseguire per: analisi dataset
"""
dataset_with_ambiti_processi = cleaned_original_dataset.copy()
for AP in ambiti_processi:
    dataset_with_ambiti_processi[AP] = 0.0

Col codice seguente, ogni studente sarà caratterizzato da un valore per ogni ambito e processo corrispondente alla proporzione di domande che vi si riferivano e a cui ha risposto correttamente: ad esempio, nel caso un processo `x` vada a caratterizzare 10 domande e lo studente risponda correttamente a 5 di queste ultime, allora il valore che quello studente presenterà sotto il processo `x` sarà pari a 5/10 = 0.5.

In [ ]:
"""
Eseguire per: rimozione feature sulle domande e aggiunta feature sugli ambiti e processi
"""
questions_columns = [col for col in list(cleaned_original_dataset) if re.search("^D\d", col)]

for i, row in dataset_with_ambiti_processi.iterrows():
    for question, APs in MAPPING_DOMANDE_AMBITI_PROCESSI.items():
        if row[question] is True:   # se ha risposto correttamente
            for AP in APs:
                dataset_with_ambiti_processi.at[i, AP] += 1 / conteggio_ambiti_processi[AP]

dataset_ap = dataset_with_ambiti_processi.drop(questions_columns, axis=1)

dataset_ap.to_csv(cfg.CLEANED_DATASET_WITH_AP)

In [ ]:
"""
Eseguire per: rimozione feature sulle domande e aggiunta feature sugli ambiti e processi
Attenzione: inutile eseguire se si è eseguita la precedente cella.
"""
dataset_ap = pd.read_csv(cfg.CLEANED_DATASET_WITH_AP)

In [ ]:
if "Unnamed: 0" in dataset_ap.columns:
    dataset_ap.drop("Unnamed: 0", axis=1, inplace=True)

## 2.6 - Analisi della correlazione fra feature

In [ ]:
corr_matrix = dataset_ap.corr(method='pearson').round(2)
corr_matrix.style.background_gradient(cmap='YlOrRd')

Emerge un'alta correlazione, 0.87, tra i voti della stessa materia, come prevedibile, mentre una correlazione abbastanza alta tra materie differenti, 0.75. La correlazione pari a 1.0 tra `pu_ma_gr` e `pu_ma_no` si spiega considerando che la seconda è la normalizzazione del valore della prima, per cui portano esattamente la stessa informazione.

Abbiamo ritenuto interessante indagare la correlazione sussistente tra i voti agli scritti e agli orali, i punteggi finali ottenuti al test e gli ambiti e i processi: abbiamo previsto solo valori positivi di correlazione, in quanto a voti maggiori, corrispondono punteggi finali maggiori, come anche per i processi e ambiti. 

In [ ]:
interesting_to_check_if_correlated_columns = [
    # Alta correlazione fra voti della stessa materia, abbastanza correlate fra materie diverse
    "voto_scritto_ita",
    "voto_orale_ita",
    "voto_scritto_mat",
    "voto_orale_mat",
    # Correlazione totale, abbastanza correlate con voti
    "pu_ma_no",
    # Target columns
    "LIVELLI",
    "DROPOUT"
] + list(ambiti_processi)

In [ ]:
check_corr_dataset = dataset_ap[interesting_to_check_if_correlated_columns].corr(method='pearson').round(2)
check_corr_dataset.style.background_gradient(cmap='YlOrRd')

La nostra previsione è stata confermata.

## 2.7 - Analisi dei tipi e gestione delle colonne

In [ ]:
print("Lista colonne e tipi:")

table = BeautifulTable()
table.columns.header = ["", "Type"]

for col in dataset_ap.columns :
    table.rows.append([col, dataset_ap[col].dtypes])
        
table.columns.alignment = BeautifulTable.ALIGN_LEFT
table.set_style(BeautifulTable.STYLE_SEPARATED)
print(table)

In [ ]:
# Le colonne DROPOUT e LIVELLI non sono considerate fra le feature in quanto colonne target (in particolare, DROPOUT è una regressione di LIVELLI).
continuous_features = columns_low_ratio_null_values + \
                      ["pu_ma_gr", "pu_ma_no", "Fattore_correzione_new", "Cheating", "WLE_MAT", "WLE_MAT_200", "WLE_MAT_200_CORR",
                       "pu_ma_no_corr"] + \
                      list(ambiti_processi) # Feature sui voti, feature elencate, ambiti e processi
if cfg.FILL_NAN == "remove":
    continuous_features.remove("voto_scritto_ita")
    continuous_features.remove("voto_orale_ita")
ordinal_features = ["n_stud_prev", "n_classi_prev"]
int_categorical_features = [
    "CODICE_SCUOLA", "CODICE_PLESSO", "CODICE_CLASSE", "campione", "prog",
]
str_categorical_features = [
    "sesso", "mese", "anno", "luogo", "eta", "freq_asilo_nido", "freq_scuola_materna",
    "luogo_padre", "titolo_padre", "prof_padre", "luogo_madre", "titolo_madre", "prof_madre",
    "regolarità", "cittadinanza", "cod_provincia_ISTAT", "Nome_reg",
    "Cod_reg", "Areageo_3", "Areageo_4", "Areageo_5", "Areageo_5_Istat"
]
bool_features = ["Pon"]

## 2.8 - Gestione dei valori nulli
Il dataset in considerazione presenta molti valori nulli.\
Vi sono diverse tecniche per gestirli, tra cui:
- sostituzione del valore nullo con un indice di sintesi (media, mediana) della colonna in considerazione: la scelta dell'indice dipende da vari fattori, tra cui la forma della distribuzione del certo attributo;
- rimozione della riga corrispondente: viene adottata solitamente quando la colonna presenta pochi valori nulli e/o la riga presenta molti valori nulli;
- rimozione della colonna corrispondente: viene adottata solitamente quando la colonna presenta un alto numero di valori nulli.

In [ ]:
dataset_ap["sigla_provincia_istat"].fillna(value="ND", inplace=True)

if cfg.FILL_NAN == "remove":
    # Rimuovere colonne voti ita (molti valori nulli).
    # Rimuovere record con dati nulli in voti mat (meno valori nulli).
    dataset_ap.drop(["voto_scritto_ita", "voto_orale_ita"], axis=1, inplace=True)
    dataset_ap.dropna(subset=["voto_scritto_mat", "voto_orale_mat"], inplace=True)
else :
    for col in columns_low_ratio_null_values : 
        if cfg.FILL_NAN == "median":
            replaced_value = dataset_ap[col].median()
        elif cfg.FILL_NAN == "mean":
            replaced_value = dataset_ap[col].mean()

        dataset_ap[col].fillna(value=replaced_value, inplace=True)   

# 3 - Machine Learning
## 3.1 - Suddivisione dataset in training e test
Avendo un dataset di una sola coorte di studenti relativi alle prove di un anno, non si dispone di un set per fare testing: Idealmente, l'insieme di testing dovrebbe referirsi ad una coorte diversa da quella su cui è stato effettuato il training. Nel caso di specie, tuttavia, si dispone dei dati relativi ad un'unica coorte, per cui abbiamo pensato di eseguire lo split in questo modo:

1. dataset diviso in training set (default 80%) e test set (default 20%);
2. training set (ottenuto dalla suddivisione al punto 1) diviso in training set (default 80%) e validation set (default 20%).

In [ ]:
df_training_set, df_test_set = train_test_split(dataset_ap, test_size=cfg.TEST_SET_PERCENT, random_state=19)

## 3.2 - Analisi dello sbilanciamento del dataset

In [ ]:
nr_nodrop, nr_drop = np.bincount(dataset_ap['DROPOUT'])
total_records = nr_drop + nr_nodrop
nl = '\n'
print(
    f"Total number of records: {total_records}{nl}\
        {nl}\
Total num. DROPOUT: {nr_drop}{nl}\
Total num. NO DROPOUT: {nr_nodrop}{nl}\
    {nl}\
Ratio DROPOUT/TOTAL: {round(nr_drop / total_records, 2)}{nl}\
Ratio NO DROPOUT/TOTAL: {round(nr_nodrop / total_records, 2)}{nl}\
    {nl}\
Ratio DROPOUT/NO DROPOUT: {round(nr_drop / nr_nodrop, 2)}"
)

Le due classi (i.e. target del classificatore) appaiono leggermente sbilanciate: in dettaglio, la classe dei soggetti che manifestano dropout ha una cardinalità inferiore della classe in cui non si è avuto dropout.

### 3.2.1 - Gestione dello sbilanciamento

Vi sono diverse tecniche per gestire lo sbilanciamento tra le classi di un attributo, tra cui:
- **ricampionamento dei dati**: 
    - *random over sampling*: selezionare randomicamente delle istanze della classe sotto rappresentata e duplicarle fino a quando le cardinalità delle classi si equivalgono; aumenta il rischio di overfitting;
    - *random under sampling*: rimuovere istanze della classe sovrarappresentata fintanto che le cardinalità delle classi si equivalgono; causa una riduzione del training set;
    - *cluster-based over-sampling*: esecuzione dell'algoritmo *k-means* sulle istanze della classe maggiormente rappresentata e su quelle della classe meno rappresentata in modo indipendente, per poi compiere oversampling sui cluster ottenuti fin tanto che le cardinalità dei cluster di una stessa classe si equivalgono come anche le cardinalità delle classi nel loro complesso;
- **generazione di dati sintetici**: 
    - *SMOTE* (Synthetic Minority Over-sampling Technique): selezionare due o più istanze simili della classe sotto rappresentata e modificare leggermente il valore di un attributo alla volta di un ammontare inferiore alla differenza tra le istanze simili; evita l'overfitting (a patto che vi siano poche attributi) ma aumenta il rumore;
- **cambiare la natura del problema**: da classificazione a *anomaly detection* o *change detection*;
- **penalizzazione delle classificazioni errate sulla classe sottorappresentata**: *Cost-sensitive Training*;
- **monitorare metriche diverse dall'accuratezza**, in quanto solitamente si ottengono ottimi valori di accuratezza con dati sbilanciati, perché il modello classifica tutti gli input come appartenenti alla classe più numerosa.

Alla luce di queste tecniche e considerate le peculiarità del nostro dataset emerse durante la EDA, abbiamo ritenuto che la tecnica migliore al caso nostro fosse il *random under sampling*: il nostro dataset presenta un alto numero di istanze e le classi dell'attributo target presentano un sbilanciamento poco accentuato per cui conviene ricorrere ad un sottocampionamento randomico, evitando così il rischio di overfitting, piuttosto che al sovracampionamento.




In [ ]:
"""
Eseguire per: campionamento
"""
if cfg.SAMPLING_TO_PERFORM == "random_undersampling":
    # class_nodrop contiene i record della classe sovrarappresentata, ovvero SENZA DROPOUT.
    class_nodrop = df_training_set[df_training_set['DROPOUT'] == False]
    # class_drop contiene i record della classe sottorappresentata, ovvero CON DROPOUT.
    class_drop = df_training_set[df_training_set['DROPOUT'] == True]

    # Sotto campionamento di class_drop in modo che abbia stessa cardinalità di class_nodrop.
    class_nodrop = class_nodrop.sample(len(class_drop), random_state=19)

    print(f'Class NO DROPOUT: {len(class_nodrop):,}')
    print(f'Classe DROPOUT: {len(class_drop):,}')

    df_training_set = class_drop.append(class_nodrop)
    df_training_set = df_training_set.sample(frac=1, random_state=19)
else:
    categorical_features_indexes = [i for i in range(len(df_training_set.columns)) if
                                    df_training_set.columns[i] in str_categorical_features + int_categorical_features]

    df_training_set = df_training_set.apply(lambda col: pd.factorize(col)[0] if col.name in str_categorical_features else col)
    df_test_set = df_test_set.apply(lambda col: pd.factorize(col)[0] if col.name in str_categorical_features else col)

    sm = SMOTENC(categorical_features=categorical_features_indexes, random_state=19)
    X_train, y_train = sm.fit_resample(
        df_training_set[[col for col in df_training_set.columns if col != 'DROPOUT']],
        df_training_set['DROPOUT']
    )
    df_training_set = pd.concat([X_train, y_train], axis=1)

    X_test, y_test = sm.fit_resample(
        df_test_set[[col for col in df_test_set.columns if col != 'DROPOUT']],
        df_test_set['DROPOUT']
    )
    df_test_set = pd.concat([X_test, y_test], axis=1)

    # Se SMOTENC viene eseguito, ogni feature categorica stringa viene trasformata in feature categorica intera.
    int_categorical_features = int_categorical_features + str_categorical_features
    str_categorical_features = []

Nonostante la scelta di orientarci verso il *random under sampling* abbiamo inserito il codice per poter eseguire l'*over sampling* mediante SMOTENC. Maggiori dettagli su questa tecnica sono presentati nella sezione [Risultati ottenuti](#risultati).

Ecco che il training set è stato bilanciato.

In [ ]:
if "Unnamed: 0" in df_training_set.columns:
    df_training_set.drop("Unnamed: 0", axis=1, inplace=True)

## 3.3 - Preprocessing per creazione del modello di Deep Learning

### 3.3.1 - Suddivisione del dataset in training e validation

In [ ]:
df_training_set, df_validation_set = train_test_split(df_training_set, test_size=cfg.VALIDATION_SET_PERCENT, random_state=19)

### 3.3.2 - Conversione dei dati da DataFrame (Pandas) a Dataset (Tensorflow/Keras)

In [ ]:
def convert_dropout_column_to_one_hot(dropout_col):
    dropout_col_one_hot = []
    for dc in dropout_col:
        if dc == 1:
            dropout_col_one_hot.append([1, 0])
        else:
            dropout_col_one_hot.append([0, 1])
    return dropout_col_one_hot

In [ ]:
def pd_dataframe_to_tf_dataset(dataframe: pd.DataFrame):
    copied_df = dataframe.copy()
    if cfg.PROBLEM_TYPE == "classification":
        dropout_col = copied_df.pop("DROPOUT")
        dropout_col = convert_dropout_column_to_one_hot(dropout_col)
        copied_df.drop("LIVELLI", axis=1, inplace=True)
    else:
        # La colonna target LIVELLI viene presa, invertiti i valori (0 -> 5, 1 -> 4, ..., 5 -> 0) e poi li divido per 5.
        # Questo viene fatto per poter associare a valori sopra 0.6 (corrispondente all'originale 4) il concetto di
        # "Dropout Sì" e a quelli inferiori il concetto di "Dropout no".
        dropout_col = copied_df.pop("LIVELLI")
        dropout_col = dropout_col.subtract(5)
        dropout_col = dropout_col.abs()
        dropout_col = dropout_col.divide(5) # Normalizzazione dei valori della colonna da [0..5] a [0..1].
        copied_df.drop("DROPOUT", axis=1, inplace=True)

    """
    Dato che il dataframe ha dati eterogenei lo convertiamo a dizionario (i.e. dict(copied_df)),
    in cui le chiavi sono i nomi delle colonne e i valori sono i valori della colonna.
    Infine bisogna indicare la colonna target.
    """

    tf_dataset = tf.data.Dataset.from_tensor_slices((dict(copied_df), dropout_col))
    tf_dataset = tf_dataset.shuffle(buffer_size=len(copied_df), seed=19)
    return tf_dataset

In [ ]:
ds_training_set = pd_dataframe_to_tf_dataset(df_training_set)
ds_validation_set = pd_dataframe_to_tf_dataset(df_validation_set)
ds_test_set = pd_dataframe_to_tf_dataset(df_test_set)

### 3.3.3 - Suddivisione del dataset in batch

In [ ]:
ds_training_set = ds_training_set.batch(cfg.BATCH_SIZE, drop_remainder=True)
ds_validation_set = ds_validation_set.batch(cfg.BATCH_SIZE, drop_remainder=True)
ds_test_set = ds_test_set.batch(cfg.BATCH_SIZE, drop_remainder=True)

### 3.3.4 - Creazione dei layer di input per ogni feature

In [ ]:
input_layers = {}
for name, column in df_training_set.items():
    if name in ["DROPOUT", "LIVELLI"]:
        continue

    if name in continuous_features:
        dtype = tf.float32
    elif name in ordinal_features or name in int_categorical_features or name in bool_features:
        dtype = tf.int64
    else:  # name in str_categorical_features
        dtype = tf.string

    input_layers[name] = tf.keras.Input(shape=(), name=name, dtype=dtype)

### 3.3.5 - Encoding delle feature in base al loro tipo

In [ ]:
preprocessed_features = []

def stack_dict(inputs, fun=tf.stack):
    values = []
    for key in sorted(inputs.keys()):
        values.append(tf.cast(inputs[key], tf.float32))

    return fun(values, axis=-1)

In [ ]:
# Preprocessing colonne con dati booleani
for name in bool_features:
    inp = input_layers[name]
    inp = inp[:, tf.newaxis]
    float_value = tf.cast(inp, tf.float32)
    preprocessed_features.append(float_value)

In [ ]:
# Preprocessing colonne con dati interi ordinali
ordinal_inputs = {}
for name in ordinal_features:
    ordinal_inputs[name] = input_layers[name]

normalizer = Normalization(axis=-1)
normalizer.adapt(stack_dict(dict(df_training_set[ordinal_features])))
ordinal_inputs = stack_dict(ordinal_inputs)
ordinal_normalized = normalizer(ordinal_inputs)
preprocessed_features.append(ordinal_normalized)

In [ ]:
# Preprocessing colonne con dati continui
continuous_inputs = {}
for name in continuous_features:
    continuous_inputs[name] = input_layers[name]

normalizer = Normalization(axis=-1)
normalizer.adapt(stack_dict(dict(df_training_set[continuous_features])))
continuous_inputs = stack_dict(continuous_inputs)
continuous_normalized = normalizer(continuous_inputs)
preprocessed_features.append(continuous_normalized)

Per i dati categorici di tipo stringa o intero abbiamo adottato la *one-hot encoding*; tale codifica consiste nel mappare ogni categoria ad un array contenente $n$-1 elementi settati a 0 e un elemento settato a 1, con $n$ pari al numero delle categorie: l'indice dell'elemento settato ad 1 permette di distinguere le diverse categorie.

In [ ]:
# Preprocessing colonne con dati categorici stringa
for name in str_categorical_features:
    vocab = sorted(set(df_training_set[name]))

    lookup = StringLookup(vocabulary=vocab, output_mode='one_hot')

    x = input_layers[name][:, tf.newaxis]
    x = lookup(x)

    preprocessed_features.append(x)

In [ ]:
# Preprocessing colonne con dati categorici interi
for name in int_categorical_features:
    vocab = sorted(set(df_training_set[name]))

    lookup = IntegerLookup(vocabulary=vocab, output_mode='one_hot')

    x = input_layers[name][:, tf.newaxis]
    x = lookup(x)

    preprocessed_features.append(x)

## 3.4 - Assemblaggio dei vari layer e creazione del modello

### 3.4.1 - Funzioni di attivazione
Abbiamo studiato e approfondito le principali funzioni di attivazione, passandone in rassegna le peculiarità, i pro e i contro. Alla luce di tali nuove conoscenze, abbiamo optato per le seguenti scelte:
- *Leaky ReLU* per gli hidden layer: 
    - abbiamo preferito la variante *Leaky ReLU* piuttosto che la classica ReLU, in quanto la prima, prevedendo una leggera pendenza a sinistra dell'origine, evita il *Dying ReLU Problem*, di cui soffre invece la ReLU;
    - il codominio ha cardinalità infinita (dato che per input > 0, diventa la funzione di identità), per cui non vi è una significativa perdita di informazione nel passaggio da un layer al successivo;
    - a fronte di somme pesate negative in input, restituisce valori negativi molto bassi che portano a calcoli non pesanti computazionalmente, registrando così tempi minori di addestramento per il modello nel complesso.
![](./src/img/leaky_relu.png)
- *Sigmoid* o *Softmax* per l'output layer: sono le funzione che vengono [consigliate](https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/) come *best practice* per il livello di output rispettivamente in problemi di regressione vista come classificazione binaria e classificazione (multi-classe).

### 3.4.2 - Inizializzazione dei pesi
L'inizializzazione dei pesi è un aspetto molto importante nella progettazione di una rete neurale: essi incidono in larga misura sulla somma pesata che si accumula in ogni neurone e che viene passata in input alla sua funzione di attivazione, determinando anche la misura in cui l'informazione passa da un layer al successivo.

I modelli di reti neurali vengono addestrati mediante l'algoritmo di ottimizzazione noto come **discesa del gradiente stocastico**, che modifica iterativamente i pesi della rete con l'obiettivo di minimizzare una funzione di costo: alla fine dell'addestramento, si giunge a quella combinazione di pesi che permettono alla rete di avere una certa performance nelle previsioni.
L'algoritmo di ottimizzazione richiede un punto di partenza nello spazio dei possibili valori dei pesi e spesso la sua velocità di convergenza o effettiva convergenza è fortemente determinata dall'inizializzazione iniziale dei pesi.

In letteratura, si suggeriscono diverse funzioni di inizializzazione a seconda della funzione di attivazione del layer in oggetto:
- con ReLU o Leaky ReLU, è consigliato il metodo di inizializzazione **He** che calcola i pesi a partire dalla distribuzione di probabilità gaussiana con media 0 e deviazione standard pari a $\sqrt{\frac{2}{n}}$, con $n$ il numero di input che arrivano al neurone;
    - conseguentemente, l'altezza della gaussiana dipende dal numero di input che il neurone riceve: all'aumentare del numero di questi input, l'altezza aumenta e la gaussiana si restringe, diminuendo la probabilità di avere pesi alti. Così i neuroni tendono meno a saturare, permettendo alla rete di raggiungere il massimo dell'accuratezza in meno tempo. Infatti, tale accorgimento non migliora l'accuratezza, ma solo il tempo che la rete impiega a raggiungere quella massima;
    
    [<img src="./src/img/He.png" width="50%"/>](https://alessiomorselli.github.io/DeepLearning/web/pages/documentation/gaussian2.png)
- con Sigmoid o Tanh, inizializzazione **Glorot** (anche nota *Xavier*, dal nome di *Xavier Glorot*, un ricercatore presso Google DeepMind): questo metodo, per ogni neurone, calcola i diversi pesi da una distribuzione di probabilità uniforme che ha come dominio l'intervallo $[-\frac{1}{\sqrt{n}}, \frac{1}{\sqrt{n}}]$, con $n$ il numero di input che arrivano al neurone considerato;
    - l'immagine seguente mostra il restringersi del range in cui cadono tali pesi all'aumentare del numero di input entranti nel neurone;
    
    [<img src="./src/img/Xavier.png" width="50%"/>](https://machinelearningmastery.com/wp-content/uploads/2021/01/Plot-of-Range-of-Xavier-Weight-Initialization-with-Inputs-from-One-to-One-Hundred-.png)
    - esiste anche una variante di tale metodo, chiamata **Normalized Xavier Weight Initialization**: essa estrae i valori da una distribuzione di probabilità uniforme, tuttavia il suo dominio è l'intervallo $[-\frac{\sqrt{6}}{\sqrt{n + m}}, \frac{\sqrt{6}}{\sqrt{n + m}}]$, con $n$ il numero di input al nodo in oggetto, $m$ il numero di output del layer di cui fa parte quel neurone (ossia il numero di neuroni di quel layer).
    
    [<img src="./src/img/Normalized_Xavier.png" width="50%"/>](https://machinelearningmastery.com/wp-content/uploads/2021/01/Plot-of-Range-of-Normalized-Xavier-Weight-Initialization-with-Inputs-from-One-to-One-Hundred.png)

    

In [ ]:
preprocessed = tf.concat(preprocessed_features, axis=-1) # Tensore

preprocessor = tf.keras.Model(input_layers, preprocessed) # Tanti input layer quante le feature

# inizializzatore che verrà usato per i pesi dei layer con ReLU / LeakyReLU
initializer_hidden_layer = tf.keras.initializers.HeNormal(seed=19)
# inizializzatore che verrà usato per i pesi dei layer con sigmoid
initializer_output_layer = tf.keras.initializers.GlorotNormal(seed=19)

### 3.4.3 - Regolarizzazione per evitare overfitting
Si ha **overfitting** ogniqualvolta il modello apprende i dati troppo bene, ossia ne incamera il rumore statistico: conseguentemente, il modello presenta un *alto errore di generalizzazione* (scarse performance quando il modello è valutato su dati nuovi, come quelli dell'insieme di test).
Il rischio di overfitting si fa più concreto quando la cardinalità del training set è bassa e/o la rete neurale è molto grande.

Per ridurre al minimo il rischio di overfitting, vi sono molti metodi di **regolarizzazione**, tra cui il **dropout**: esso approssima l'addestramento in parallelo di un gran numero di reti neurali con architetture differenti; consiste nell'assegnare ad ogni neurone di un certo layer una probabilità $1-p$ che sia ignorato e $p$ che non sia ignorato, per cui si concretizza in un sottocampionamento degli output dei vari layer.
La probabilità $p$ viene settato a livello del singolo layer; in letteratura, sono suggeriti valori prossimi a 0.5 per i layer nascosti e prossimi a 1 per i layer visibili, mentre di non settare alcun dropout per il layer di output. Noi abbiamo rispettato tali suggerimenti, come è possibile constatare nel codice che segue.

Un effetto collaterale del dropout è la riduzione della capacità rappresentativa della rete, per cui è conveniente solo a fronte di reti neurali grandi: nel caso la rete abbia dimensioni modeste, occorre aumentarne prima l'ampiezza e la profondità e poi settare il dropout ai vari layer. Questo motivo è alla base della nostra scelta di raddoppiare i valori di default degli iperparametri relativi al numero di layer e al numero di neuroni nelle configurazioni in cui è presente il dropout. 


### 3.4.4 - Batch Normalization
Tra le innumerevoli architetture esperite, ve ne sono state alcune con un alto numero di hidden layer, nell'ordine delle decine; avendo riscontrato significativi rallentamenti nell'addestramento, abbiamo effettuato diverse ricerche in seguito alle quali abbiamo scoperto la **batch normalization**: trattasi di una tecnica che standardizza gli input al layer cui è applicata per ogni mini-batch di addestramento. In questo modo, i valori in input sono minori e centrati, quindi manipolabili in maniera più agevole: conseguentemente il processo di apprendimento risulta essere più stabile e veloce, impiegando un numero di epoche inferiore.\
Gli studi a tal proposito in letteratura suggeriscono di sperimentare l'applicazione di tale processo di standardizzazione in tre punti differenti:
- tra l'hidden layer e la sua funzione di attivazione;
- dopo la funzione di attivazione di un hidden layer;
- subito prima del layer di output. 

In [ ]:
body = tf.keras.Sequential()

if cfg.DROPOUT_LAYER:
    body.add(tf.keras.layers.Dropout(rate=cfg.DROPOUT_INPUT_LAYER_RATE, seed=19))  # aggiunta dropout a layer di input

# segue l'aggiunta degli hidden layers
for _ in range(cfg.NUMBER_OF_LAYERS):
    body.add(tf.keras.layers.Dense(cfg.NEURONS, kernel_initializer=initializer_hidden_layer))

    if cfg.BATCH_NORMALIZATION == "dense_batch_activation":
        body.add(tf.keras.layers.BatchNormalization())

    if cfg.ACTIVATION_LAYER == "leaky_relu":
        body.add(tf.keras.layers.LeakyReLU())
    else:
        body.add(tf.keras.layers.ReLU())

    if cfg.DROPOUT_LAYER:
        body.add(tf.keras.layers.Dropout(rate=cfg.DROPOUT_HIDDEN_LAYER_RATE, seed=19))

    if cfg.BATCH_NORMALIZATION == "dense_activation_batch":
        body.add(tf.keras.layers.BatchNormalization())

if cfg.BATCH_NORMALIZATION == "before_output":
    body.add(tf.keras.layers.BatchNormalization())

# segue l'aggiunta dell'output layer
if cfg.PROBLEM_TYPE == "classification":
    body.add(tf.keras.layers.Dense(2, activation="softmax", kernel_initializer=initializer_output_layer))
else:
    body.add(tf.keras.layers.Dense(1, activation="sigmoid", kernel_initializer=initializer_output_layer))

x = preprocessor(input_layers)

result = body(x)

model = tf.keras.Model(input_layers, result)

In [ ]:
# Visualizzazione tabellare del modello
from keras.utils.vis_utils import plot_model
model.summary()
plot_model(model, show_shapes=True, show_layer_names=True)

### 3.4.5 - Funzione di costo e metriche di performance
Riguardo alla funzione di costo, abbiamo scelto per il nostro modello la **Cross-entropy**: come riportato dal libro "Pattern Recognition and Machine Learning" (2006), utilizzare la cross-entropy piuttosto che la somma dei quadrati come funzione di costo nei problemi di classificazione porta ad un apprendimento più veloce e ad una generalizzazione maggiore.\
Trattasi di una misura derivante dal campo della *Teoria dell'informazione*: nella sua accezione più generale, si concretizza nel calcolare la differenza (in termini più precisi, l'entropia totale) tra due distribuzioni di probabilità, dato una variabile aleatoria o un insieme di eventi. Nell'ambito del *Machine Learning*, tali due distribuzioni sono:
- la distribuzione della variabile target e
- l'approssimazione della distribuzione della variabile target restituita dal predittore.

Pertanto, la cross-entropy è un valore positivo indicante l'entropia totale necessaria affinchè i valori della variabile target siano rappresentati/trasmessi mediante la distribuzione approssimata delle predizioni.

[<img src="./src/img/cross-entropy.png" width="50%"/>](https://i.stack.imgur.com/gNip2.png)

Più le due distribuzioni divergono, maggiore è la cross-entropy: ciò è dovuto all'informazione ( = $-\log_2{P(x)}$) maggiore a fronte di predizioni molto distanti da 1.

[<img src="./src/img/information.png" width="50%"/>](https://i.stack.imgur.com/gNip2.png)

A livello implementativo, nel caso il problema in esame sia visto come un problema di:
    
- **regressione**, l'output compreso nell'intervallo $[0, 1]$ viene valutato con la funzione di costo `BinaryCrossentropy()` poiché ci interessa sapere solamente quando l'output è inferiore del threshold fissato (0.4) oppure maggiore;
- **classificazione**, l'output è un vettore con tante componenti quante le classi di output (in questo caso due, "dropout sì" e "dropout no"), in cui ogni componente è una misura di probabilità di far parte di quella classe, il tutto valutato con la funzione di costo `CategoricalCrossEntropy()`.

In [ ]:
if cfg.PROBLEM_TYPE == "classification":
    accuracy = tf.keras.metrics.Accuracy(name="acc")
    loss_function = tf.keras.losses.CategoricalCrossentropy()
else:
    # 0.6 perché dopo il preprocessing, i LIVELLI in [3,4,5] è DROPOUT = True, LIVELLI in [0,1,2] è DROPOUT = False
    accuracy = tf.metrics.BinaryAccuracy(name="bin_acc", threshold=0.6)  
    loss_function = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=cfg.LEARNING_RATE),
              loss=loss_function,
              metrics=[
                accuracy,
                tf.keras.metrics.FalsePositives(name="fp"),
                tf.keras.metrics.FalseNegatives(name="fn"),
                tf.keras.metrics.TruePositives(name="tp"),
                tf.keras.metrics.TrueNegatives(name="tn"),
                tf.keras.metrics.Precision(name="prec"),
                tf.keras.metrics.Recall(name="rec")
              ])

### 3.4.6 - Early stopping
Mano a mano che procedavameno nello sperimentare differenti architetture, abbiamo riscontrato qualche dubbio circa il numero giusto di epoche per l'apprendimento\.
In effetti, ricerche a tal proposito, hanno evidenziato che trattasi di un iperparametro il cui settaggio è di fondamentale e delicata importanza; in dettaglio, settare:
- un numero troppo elevato di epoche, può portare al rischio dell'overfitting del modello sul training set mentre,
- un numero troppo basso può essere la causa di un apprendimento troppo grossolano degli schemi celati dietro ai dati.

Uno dei metodi suggeriti in letteratura è il c.d. **Early stopping**: esso permette di specificare un numero di epoche arbitrariamente alto, in quanto gestisce l'effettiva fine dell'addestramento sulla base delle variazioni registrate da una certa metrica specificata. 
Nel caso del nostro progetto abbiamo specificato come metrica da monitorare il valore restituito dalla funzione di costo sul validation set.\
Col parametro `mode` specifichiamo la direzione della variazione della metrica da considerarsi come miglioramento.
Per evitare che l'addestramento venga stoppato non appena il valore della funzione di costo non migliora (nel caso di specie, migliora sta per diminuisce), abbiamo settato un ulteriore argomento, `patience` che indica il numero di epoche che occorre attendere prima di fermare l'addestramento in caso non ci siano miglioramenti; così facendo, possiamo contemplare la situazione in cui la metrica monitorata sperimenti un plateau o addirittura un peggioramento (nel caso di specie, sta per aumento) per qualche epoca, prima di tornare a migliorare.\
Ulteriormente abbiamo settato a `True` il parametro `restore_best_weight`, allo scopo di ripristinare i valori dell'ultima epoca in cui la metrica monitorata è migliorata: questo permette di non evitare che la rete rimanga coi pesi dell'ultima epoca fin cui si è addestrata, grazie al parametro `patience`.

In [ ]:
"""
Definizione dello stopper per evitare che la reti continui ad addestrarsi quando non ci sono miglioramenti della loss 
(val_loss = funzione di costo sul validation set) per piu' di 5 epoche
"""
early_stopper = EarlyStopping(monitor="val_loss",
                              patience=5,
                              mode="min",
                              restore_best_weights=True)

### 3.4.7 - Training

In [ ]:
print("[Training]")
history = model.fit(ds_training_set,
          epochs=cfg.EPOCH,
          batch_size=cfg.BATCH_SIZE,
          validation_data=ds_validation_set,
          callbacks=([early_stopper] if cfg.EARLY_STOPPING else []) + [PlotLossesKeras()],
          verbose=False)

### 3.4.8 - Test

In [ ]:
print("[Test]")
score = model.evaluate(ds_test_set)

In [ ]:
print('Results with test dataset')
print('Loss:', round(score[0], 4))
print('Accuracy:', round(score[1], 4))
print('False positives:', int(score[2]))
print('False negatives:', int(score[3]))
print('True positives:', int(score[4]))
print('True negatives:', int(score[5]))
print('Precision: ', round(score[6], 4))
print('Recall: ', round(score[7], 4))

### 3.4.9 - Calcolo manuale delle metriche a partire dalla matrice di confusione

In [ ]:
# Selezione delle colonne delle feature (utilizzo di dataset_ap, ma uno qualsiasi fra df_training_set,
# df_test_set e df_validation_set andava bene lo stesso)
feature_columns = dataset_ap[[col for col in dataset_ap.columns if col not in ["DROPOUT", "LIVELLI"]]]
target_col = "DROPOUT" if cfg.PROBLEM_TYPE == "classification" else "LIVELLI"

In [ ]:
def convert_df_for_prediction(dataframe: pd.DataFrame):
    copied_df = dataframe.copy()
    ds = tf.data.Dataset.from_tensor_slices(dict(copied_df))

    return ds.batch(cfg.BATCH_SIZE, drop_remainder=True)


def convert_for_confusion_matrix(dataframe: pd.DataFrame):
    X = convert_df_for_prediction(dataframe[feature_columns])
    y = dataframe[target_col]
    len_X = len(X) * cfg.BATCH_SIZE
    len_y = len(y)
    
    if len_X != len_y:
        y = y.head(len_X - len_y)
    if cfg.PROBLEM_TYPE == "classification":
        y = convert_dropout_column_to_one_hot(y)
    else: # cfg.PROBLEM_TYPE == "regression"
        y = y.subtract(5)
        y = y.abs()
    
    return X, y

In [ ]:
training_x, training_y = convert_for_confusion_matrix(df_training_set)
validation_x, validation_y = convert_for_confusion_matrix(df_validation_set)
test_x, test_y = convert_for_confusion_matrix(df_test_set)

In [ ]:
predicted_training_y = model.predict(training_x)
predicted_validation_y = model.predict(validation_x)
predicted_test_y = model.predict(test_x)

if cfg.PROBLEM_TYPE == "regression":
    predicted_training_y = np.round(predicted_training_y * 5)
    predicted_validation_y = np.round(predicted_validation_y * 5)
    predicted_test_y = np.round(predicted_test_y * 5)

In [ ]:
training_confusion_matrix = tf.math.confusion_matrix(labels=training_y, predictions=predicted_training_y).numpy()
validation_confusion_matrix = tf.math.confusion_matrix(labels=validation_y, predictions=predicted_validation_y).numpy()
test_confusion_matrix = tf.math.confusion_matrix(labels=test_y, predictions=predicted_test_y).numpy()

In [ ]:
def compute_metrics(label, confusion_matrix):
    true_positives = np.diag(confusion_matrix) # vettore in cui ogni cella è il numero di TP per la classe
    false_positives = confusion_matrix.sum(axis=0) - true_positives
    false_negatives = confusion_matrix.sum(axis=1) - true_positives
    true_negatives = confusion_matrix.sum() - (true_positives.sum() + false_positives.sum() + false_negatives.sum())

    accuracy = (true_positives.sum() + true_negatives.sum()) / confusion_matrix.sum()
    precision = true_positives.sum() / (true_positives.sum() + false_positives.sum())
    recall = true_positives.sum() / (true_positives.sum() + false_negatives.sum())

    print(f"Confusion Matrix Name: {label}")
    print(confusion_matrix)
    print(f"- TP: {int(true_positives.sum())}")
    print(f"- TN: {int(true_negatives.sum())}")
    print(f"- FP: {int(false_positives.sum())}")
    print(f"- FN: {int(false_negatives.sum())}")
    print(f"- Accuracy: {round(accuracy, 4)}") # Attenzione: si tratta della Binary Accuracy
    print(f"- Precision: {round(precision, 4)}")
    print(f"- Recall: {round(recall, 4)}")
    print()

In [ ]:
compute_metrics("Training", training_confusion_matrix)
compute_metrics("Validation", validation_confusion_matrix)
compute_metrics("Test", test_confusion_matrix)

<a id="risultati"></a>
# 4 - Risultati ottenuti dall'esecuzione del modello

Abbiamo cercato di trovare la configurazione migliore per risolvere questo tipo di problema avviando numerosi job grazie al Cluster HPC del DISI, di cui viene trattato più in dettaglio in [Esecuzione su Cluster HPC](#cluster).

Tutte le configurazioni di iperparametri sono state testate sia vedendo il problema come regressione che con il problema visto come classificazione multi-classe.

Abbiamo fatto più iterazioni:
- la prima è servita a capire quali iperparametri fornivano i risultati più convincenti da usare nella successiva iterazione
- la seconda ha permesso di testare se configurazioni più avanzate con gli iperparametri più importanti scoperti nell'iterazione precedente fornissero risultati migliori
- la terza ci ha permesso di confermare la bontà degli iperparametri trovati alla seconda iterazione.

## 4.1 - Prima iterazione

La configurazione di default (sia per il problema trattato come classificazione che regressione) è la seguente:

Epoche | Neuroni | Batch | Layer lineari (numero, attivazione) | Tasso apprendimento | Dropout | Tecnica riempimento valori nulli | Batch normalization |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| 50 | 128 | 32 | 10, Leaky ReLU | 0.001 | No | mediana | No |

Nelle tabella che seguono vengono indicati i valori di accuratezza riscontrati e delle note (quando necessario).

**N.B.** Solo le celle che differiscono dalla configurazione di default vengono compilate.

### 4.1.1 - Classificazione



| Configurazione | Epoche | Neuroni | Batch | Layer lineari (numero, attivazione) | Tasso apprendimento | Dropout | Tecnica riempimento valori nulli | Batch normalization | Accuratezza in training | Accuratezza in validation | Accuratezza in test | Note aggiuntive |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| base_model | - | - | - | - | - | - | - | - | 0.3451 | 0.2581 | 0.2809 | - |
| batch128 | - | - | 128 | - | - | - | - | - | 0.2990 | 0.1936 | 0.2076 | - |
| neurons256 | - | 256 | - | - | - | - | - | - | 0.6569 | 0.3675 | 0.3995 | - |
| batch128neurons256 | - | 256 | 128 | - | - | - | - | - | 0.4621 | 0.3638 | 0.3744 | - |
| dense5 | - | - | - | 5, Leaky ReLU | - | - | - | - | 0.3202 | 0.1477 | 0.1645 | - |
| dense15 | - | - | - | 15, Leaky ReLU | - | - | - | - | 0.5303 | 0.3234 | 0.3038 | - |
| dense15 | - | - | - | 15, Leaky ReLU | - | - | - | - | 0.5303 | 0.3234 | 0.3038 | - |
| dropout | - | 256 | - | - | - | Sì | - | - | 0.0024 | 0.0000 | 0.0000 | - |
| epoch100 | 100 | - | - | - | - | - | - | - | 0.5000 | 0.2945 | 0.3135 | - |
| fillmean | - | - | - | - | - | - | media | - | 0.3538 | 0.2022 | 0.2070 | - |
| fillremove | - | - | - | - | - | - | rimozione | - | 0.3538 | 0.2022 | 0.2070 | Le colonne con alto tasso di valori nulli vengono rimosse, mentre quelle con un basso tasso vengono rimossi i record con quella feature a null. |
| lr001 | - | - | - | - | - | - | - | - | 0.7400 | 0.7131 | 0.6720 | - |
| relu | - | - | - | 10, ReLU | - | - | - | - | 0.4603 | 0.1242 | 0.1639 | - |

### 4.1.2 - Regressione

| Configurazione | Epoche | Neuroni | Batch | Layer lineari (numero, attivazione) | Tasso apprendimento | Dropout | Tecnica riempimento valori nulli | Batch normalization | Accuratezza in training | Accuratezza in validation | Accuratezza in test | Note aggiuntive |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| base_model | - | - | - | - | - | - | - | - | 0.1975 | 0.1897 | 0.2460 | - |
| batch128 | - | - | 128 | - | - | - | - | - | 0.1976 | 0.1908 | 0.2494 | - |
| neurons256 | - | 256 | - | - | - | - | - | - | 0.1975 | 0.1865 | 0.2418 | - |
| batch128neurons256 | - | 256 | 128 | - | - | - | - | - | 0.1976 | 0.1868 | 0.2417 | - |
| dense5 | - | - | - | 5, Leaky ReLU | - | - | - | - | 0.1975 | 0.1869 | 0.2426 | - |
| dense15 | - | - | - | 15, Leaky ReLU | - | - | - | - | 0.1975 | 0.1822 | 0.2361 | - |
| dropout | - | 256 | - | - | - | Sì | - | - | 0.1903 | 0.1964 | 0.2563 | - |
| epoch100 | 100 | - | - | - | - | - | - | - | 0.1975 | 0.1840 | 0.2380 | - |
| fillmean | - | - | - | - | - | - | media | - | 0.1975 | 0.1874 | 0.2422 | - |
| fillremove | - | - | - | - | - | - | rimozione | - | 0.1869 | 0.1771 | 0.2174 | Le colonne con alto tasso di valori nulli vengono rimosse, mentre quelle con un basso tasso vengono rimossi i record con quella feature a null. |
| lr001 | - | - | - | - | - | - | - | - | 0.1493 | 0.1162 | 0.1502 | - |
| relu | - | - | - | 10, ReLU | - | - | - | - | 0.1975 | 0.1924 | 0.2497 | - |

## 4.2 - Seconda iterazione

Analizzando le tabelle della prima iterazione osserviamo che gli iperparametri più importanti sono il numero dei neuroni, il numero di hidden layer e il learning rate.

Per questa ragione, abbiamo ideato delle configurazioni con decisamente più neuroni e learning rate più elevati, tenendo però più basso il numero di hidden layer, ottenendo i risultati che seguono. 

### 4.2.1 - Classificazione

| Configurazione | Epoche | Neuroni | Batch | Layer lineari (numero, attivazione) | Tasso apprendimento | Dropout | Tecnica riempimento valori nulli | Batch normalization | Accuratezza in training | Accuratezza in validation | Accuratezza in test | Note aggiuntive |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| 512N_001LR | - | 512 | - | - | 0.01 | - | - | - | 0.7292 | 0.6546 | 0.7322 | - |
| 512N_7L_01LR_DrTrue | - | 512 | - | 7, Leaky ReLU | 0.1 | Sì | - | - | 0.5803 | 0.5965 | 0.4735 | - |
| 512N_7L_001LR | - | 512 | - | - | 7, Leaky ReLU | 0.01 | - | - | 0.7561 | 0.6419 | 0.5438 | - |
| 512N_7L_001LR_DrTrue | - | - | - | 7, Leaky ReLU | 0.01 | Sì | - | - | 0.5782 | 0.5473 | 0.4065 | - |
| before_output | - | - | - | - | - | - | - | Prima del layer di output | $1.8430 \cdot 10^-4$ | $6.7003 \cdot 10^-5$ | $7.3082 \cdot 10^-5$ | - |
| dense_activation_batch | - | - | - | - | - | - | - | ... | 0.1437 | 0.0710 | 0.0659 | - |
| dense_batch_activation | - | - | - | - | - | - | - | ... | 0.1307 | 0.0809 | 0.0900 | - |

### 4.2.2 - Regressione

| Configurazione | Epoche | Neuroni | Batch | Layer lineari (numero, attivazione) | Tasso apprendimento | Dropout | Tecnica riempimento valori nulli | Batch normalization | Accuratezza in training | Accuratezza in validation | Accuratezza in test | Note aggiuntive |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| 512N_001LR | - | 512 | - | - | 0.01 | - | - | - | 0.1509 | 0.1962 | 0.2560 | - |
| 512N_7L_01LR_DrTrue | - | 512 | - | 7, Leaky ReLU | 0.1 | Sì | - | - | 0.1149 | 0.0033 | 0.0019 | - |
| 512N_7L_001LR | - | 512 | - | - | 7, Leaky ReLU | 0.01 | - | - | 0.1570 | 0.1396 | 0.1781 | - |
| 512N_7L_001LR_DrTrue | - | - | - | 7, Leaky ReLU | 0.01 | Sì | - | - | 0.1147 | 0.1951 | 0.2549 | - |
| before_output | - | - | - | - | - | - | - | Prima del layer di output | 0.1975 | 0.1745 | 0.2252 | - |
| dense_activation_batch | - | - | - | - | - | - | - | ... | 0.1974 | 0.1897 | 0.2464 | - |
| dense_batch_activation | - | - | - | - | - | - | - | ... | 0.1975 | 0.1882 | 0.2444 | - |

## 4.3 - Ultimi tentativi

Considerati i buoni risultati ottenuti in particolar modo dalla configurazione chiamata `512N_001LR` abbiamo provato a tenere fisso il numero di neuroni e di aumentare il numero di hidden layer. Al contempo abbiamo provato anche a tenere il learning rate a 0.01 ma il framework Tensorflow non è stato in grado di proseguire per problemi a runtime. Abbiamo quindi provato a ridurre questo tasso fino a 0.005, ovvero la metà di quello desiderato.

### 4.3.1 - Classificazione

| Configurazione | Epoche | Neuroni | Batch | Layer lineari (numero, attivazione) | Tasso apprendimento | Dropout | Tecnica riempimento valori nulli | Batch normalization | Accuratezza in training | Accuratezza in validation | Accuratezza in test | Note aggiuntive |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| great | - | 512 | - | 15, Leaky ReLU | 0.005 | - | - | - | 0.7230 | 0.7161 | 0.7138 | - |
| smotenc | - | - | - | - | - | - | - | - | ND | ND | ND | Dataset diviso in training 50%, validation 25%, test 25%, sampling eseguito con SMOTENC. |

### 4.3.2 - Regressione

| Configurazione | Epoche | Neuroni | Batch | Layer lineari (numero, attivazione) | Tasso apprendimento | Dropout | Tecnica riempimento valori nulli | Batch normalization | Accuratezza in training | Accuratezza in validation | Accuratezza in test | Note aggiuntive |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| great | - | 512 | - | 15, Leaky ReLU | 0.005 | - | - | - | 0.1535 | 0.1391 | 0.1783 | - |
| smotenc | - | - | - | - | - | - | - | - | ND | ND | ND | Dataset diviso in training 50%, validation 25%, test 25%, sampling eseguito con SMOTENC. |

## 4.4 - Conclusioni

Per quanto riguarda il problema di classificazione, nonostante l'aumento di hidden layer notiamo come la configurazione `great` non fornisca risultati particolarmente migliori rispetto a quelli forniti da quella su cui ci eravamo basati, `512N_001LR`, aumentando al contempo il costo computazionale. Possiamo quindi ritenere la configurazione `512N_001LR` come soddisfacente per la risoluzione di questo problema.

Per quanto riguarda invece il problema di regressione, .... COMPLETARE.

<a id="cluster"></a>
# 5 - Esecuzione su Cluster HPC
Per via della numerosità e complessità computazionale dei test nella fase di progettazione dell'architettura della rete, abbiamo pensato di usufruire del servizio di High Performance Computing su cluster dipartimentale con GPU offerto dal dipartimento DISI.

Dopo aver fatto richiesta, i nostri account istituzionali sono stati abilitati all'accesso ai sistemi dipartimentali e al cluster stesso: la macchina di nostro interesse è `slurm.cs.unibo.it` su cui si trova lo schedulatore del cluster. 

In dettaglio, il cluster utilizza uno schedulatore [SLURM](https://slurm.schedmd.com/overview.html) per la distribuzione 
dei job. Pertanto, per effettuare il submit di un job, abbiamo predisposto nella nostra area di lavoro un file di script 
SLURM contenente le direttive per la configurazione del job di interesse.\
Sulla scia delle [raccomandazioni](https://disi.unibo.it/it/dipartimento/servizi-tecnici-e-amministrativi/servizi-informatici/utilizzo-cluster-hpc/unibo.tiles.multi.links_attachments/e7809f6f52644346a912b99dd2280788/@@objects-download/11906a7e3d2345278c0fc5ee68ce7975/file/IstruzioniUsoClusterGPU.pdf) contenute nelle istruzioni consegnateci dai tecnici del DISI, abbiamo proceduto col creare sulla macchina slurm un virtual environment 
Python, in cui, mediante il comando `pip3 install` abbiamo installato le dipendenze necessarie (specificate nel file di testo `requirements.txt`).

Abbiamo creato differenti file di script SLURM per le diverse architetture neurali progettate. A titolo di esempio, riportiamo il contenuto dello script di default.

```bash
#!/bin/bash
#SBATCH --job-name=base_model
#SBATCH --mail-type=ALL
#SBATCH --mail-user=tommaso.azzalin@studio.unibo.it
#SBATCH --time=10:00:00
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --output=base_model
#SBATCH --gres=gpu:1

cd  ../

. venv/bin/activate # per attivare il virtual environment python

pip3 install -r requirements.txt

python3 src/base.py
```

# 6 - Lavori futuri
Abbiamo intenzione di continuare a lavorare a questo progetto, perché riteniamo sia un ottimo banco di prova per maturare nuove conoscenze e competenze relative al Machine Learning. In dettaglio, prevediamo di concentrare gli sforzi futuro lungo le seguenti direzioni:
- gestione dello sbilanciamento del training set mediante:
    - *SMOTE*, in quanto permette di evitare la riduzione di cardinalità del training set che inevitabilmente segue al *random under-sampling*, senza rischiare l'overfitting proprio del *random over-sampling*;
    - *Random Weighted Sampling*.